In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../input/fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
fd = pd.read_csv('../input/fake-news/test.csv')

In [5]:
fd.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [6]:
df.shape

(20800, 5)

In [7]:
fd.shape

(5200, 4)

In [8]:
fd.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [9]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


In [10]:
fd['title'] = fd['title'].fillna(fd['title'].mode()[0])
fd['author']= fd['author'].fillna(fd['author'].mode()[0])
fd['text'] = fd['text'].fillna(fd['text'].mode()[0])

In [11]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
fd.shape

(5200, 4)

In [13]:
df = df.dropna()

In [14]:
df.shape

(18285, 5)

In [15]:
x = df.drop('label',axis=1)

In [16]:
y = df['label']

In [17]:
x.shape

(18285, 4)

In [18]:
y.shape

(18285,)

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'2.3.1'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [22]:
# vocabulary size
voc_size = 5000

One Hot Representation

In [23]:
messages = x.copy()

In [24]:
messages2 = fd.copy()

In [25]:
messages2.shape

(5200, 4)

In [26]:
messages2.reset_index(inplace=True)

In [27]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [28]:
messages.reset_index(inplace=True)

In [29]:
import nltk
import re
from nltk.corpus import stopwords

In [30]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5200 non-null   object
 2   author  5200 non-null   object
 3   text    5200 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


In [31]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus2 = []
for i in range(0,len(messages2)):
    print(i)
    review2 = re.sub('[^a-zA-Z]',' ',messages2['title'][i])
    review2 = review2.lower()
    review2 = review2.split()
    
    review2 = [ps.stem(word) for word in review2 if not word in stopwords.words('english')]
    review2 = ' '.join(review2)
    corpus2.append(review2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
corpus2

['specter trump loosen tongu purs string silicon valley new york time',
 'russian warship readi strike terrorist near aleppo',
 'nodapl nativ american leader vow stay winter file lawsuit polic',
 'tim tebow attempt anoth comeback time basebal new york time',
 'keiser report meme war e',
 'trump usa antiqu hero clinton next presid',
 'pelosi call fbi investig find russian donald trump breitbart',
 'weekli featur profil randi shannon',
 'urban popul boom make climat chang wors',
 'poll rig trump wildli diverg survey cannot possibl correct',
 'u gener admir endors trump command chief',
 'work class hero john brennon',
 'rise mandatori vaccin mean end medic freedom',
 'communist terror small busi',
 'comput programm come forward admit paid rig vote booth trump right usa newsflash',
 'thiev take chunk chang pound berlin museum new york time',
 'new england patriot owner still sore n f l payback sight new york time',
 'colleg republican yaf sue berkeley ann coulter event breitbart',
 'trump 

In [34]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [35]:
onehot_repr2 = [one_hot(words,voc_size) for words in corpus2]
onehot_repr2

[[2628, 1687, 4394, 4963, 2980, 1187, 710, 678, 4815, 2330, 3174],
 [848, 1405, 174, 3748, 3998, 3982, 4298],
 [3270, 897, 3325, 10, 1744, 1368, 3440, 1498, 279, 3230],
 [3034, 963, 2203, 4597, 224, 3174, 4055, 4815, 2330, 3174],
 [4440, 950, 4002, 3523, 3961],
 [1687, 831, 804, 1889, 4565, 1930, 3754],
 [4897, 3917, 3344, 1544, 3986, 848, 1362, 1687, 1421],
 [2202, 772, 3177, 3799, 729],
 [1157, 1290, 2210, 360, 4837, 290, 2543],
 [1781, 4612, 1687, 2778, 2537, 627, 1023, 574, 1411],
 [512, 2317, 3491, 1200, 1687, 1844, 41],
 [3366, 4336, 1889, 2364, 3811],
 [3694, 4565, 1122, 1595, 4896, 4947, 3112],
 [1028, 153, 278, 797],
 [339, 4982, 601, 664, 4866, 3881, 4612, 915, 4355, 1687, 123, 831, 4171],
 [3425, 571, 4063, 290, 197, 2226, 4206, 4815, 2330, 3174],
 [4815,
  2415,
  112,
  3580,
  1582,
  1227,
  3140,
  98,
  321,
  1834,
  1199,
  4815,
  2330,
  3174],
 [4397, 2889, 2313, 1348, 2754, 3742, 4248, 406, 1421],
 [1687, 3283, 4517, 4221, 3689, 709, 941, 3944, 4565],
 [1735, 426

In [36]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4406, 3523, 3600, 3111, 3217, 4535, 2648, 3357, 3407, 659],
 [2848, 1831, 4565, 1542, 2722, 1623, 1421],
 [654, 2410, 772, 2308],
 [1264, 4946, 1848, 4221, 1439, 3274],
 [2474, 2722, 646, 1477, 1453, 1429, 2722, 4789, 3918, 3784],
 [4503,
  3817,
  1703,
  1597,
  4236,
  1687,
  2946,
  1656,
  2631,
  1532,
  3276,
  3546,
  1185,
  3383,
  1421],
 [3005, 2564, 2921, 2422, 2119, 3532, 1585, 3461, 4815, 2330, 3174],
 [2694, 1982, 3231, 4585, 1764, 1294, 1687, 3625, 4815, 2330, 3174],
 [3294, 3154, 1728, 3572, 2874, 1628, 3395, 4571, 1687, 1815],
 [2359, 3890, 44, 4079, 3999, 1761, 3827, 4046],
 [848, 1516, 2276, 1294, 4272, 2334, 4161, 2593, 179, 1806, 2512],
 [4221, 4003, 3217, 1628, 1687, 1764],
 [1246, 3881, 441, 3264, 1046, 4064, 3553, 1684, 2170],
 [1793, 2079, 3526, 3253, 3986, 390, 2574, 4815, 2330, 3174],
 [502, 3511, 41, 2240, 1042, 4815, 2330, 3174],
 [4403, 380, 1210, 4829, 215, 4783, 738, 1702, 3161, 2588],
 [3344, 1280, 1831],
 [1441, 1437, 1066, 1362, 1687, 887, 2842, 

Embedding representation 

In [37]:
sent_length = 20
embedded_docs2 = pad_sequences(onehot_repr2,padding='pre',maxlen=sent_length)
embedded_docs2

array([[   0,    0,    0, ..., 4815, 2330, 3174],
       [   0,    0,    0, ..., 3998, 3982, 4298],
       [   0,    0,    0, ..., 1498,  279, 3230],
       ...,
       [   0,    0,    0, ..., 4815, 2330, 3174],
       [   0,    0,    0, ...,  848, 2853,  818],
       [   0,    0,    0, ..., 4815, 2330, 3174]], dtype=int32)

In [38]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 3357, 3407,  659],
       [   0,    0,    0, ..., 2722, 1623, 1421],
       [   0,    0,    0, ..., 2410,  772, 2308],
       ...,
       [   0,    0,    0, ..., 4815, 2330, 3174],
       [   0,    0,    0, ..., 4607, 4207, 2768],
       [   0,    0,    0, ...,  817,   98, 4355]], dtype=int32)

In [39]:
len(embedded_docs)

18285

In [40]:
len(embedded_docs2)

5200

In [41]:
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
len(embedded_docs),len(embedded_docs2)

(18285, 5200)

In [43]:
y.shape

(18285,)

In [44]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [45]:
fd_final = np.array(embedded_docs2)

In [46]:
fd_final.shape

(5200, 20)

In [47]:
x_final.shape

(18285, 20)

In [48]:
y_final.shape

(18285,)

In [49]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [50]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 25ms/step - loss: 0.3397 - accuracy: 0.8377 - val_loss: 0.1977 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1467 - accuracy: 0.9429 - val_loss: 0.1958 - val_accuracy: 0.9157
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1097 - accuracy: 0.9583 - val_loss: 0.1983 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0853 - accuracy: 0.9682 - val_loss: 0.2685 - val_accuracy: 0.9195
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0670 - accuracy: 0.9767 - val_loss: 0.2566 - val_accuracy: 0.9114
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0586 - accuracy: 0.9784 - val_loss: 0.2686 - val_accuracy: 0.9062
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0477 - accuracy: 0.9833 - val_loss: 0.3212 - val_accuracy: 0.9115

In [51]:
y_predict = model.predict_classes(x_test)

In [52]:
y_predict2 = model.predict_classes(fd_final)

In [53]:
result = pd.read_csv('../input/fake-news/submit.csv')

In [54]:
result.shape

(5200, 2)

In [55]:
y_predict2 = y_predict2.flatten()

In [56]:
y_predict2_pandas = pd.DataFrame(y_predict2)

In [57]:
y_predict2_pandas.head()


,0
0,0
1,1
2,0
3,0
4,0


In [58]:
res2 = pd.concat([result,y_predict2_pandas],axis=1)

In [59]:
res2.head()

,id,label,0
0,20800,0,0
1,20801,1,1
2,20802,0,0
3,20803,1,0
4,20804,1,0


In [60]:
res2 = res2.drop('label',axis=1)

In [61]:
res2 = res2.rename(columns={0:'label'})


In [62]:
res2.shape

(5200, 2)

In [63]:
res2.to_csv('mysubmission2.csv',index=False)